# Prueba de Data Enginieer PRAGMA
\
**Realizado por: Sergio Duque Mejía**
\
**cedula: 1036654435**

En el presente Notebook se muestra la solución realizada para la prueba de acercamiento al rol de Data Enginieer, el codigo se encuentra comentado y ademas se especifica el por que se utilizo liberias y algunos pasos especificos para poder integrar desde Python3 el pipeline que permite conectar desde el lenguaje a MySQL.

### Esquema de la carpeta del proyecto:
Pragma prueba (carpeta)
\
-----| notebook_sql.ipynb (notebook)
\
-----| dataPruebaDataEngineer (carpeta)
\
----------| 2012-1.csv 
\
----------| 2012-2.csv 
\
----------| 2012-3.csv 
\
----------| 2012-4.csv 
\
----------| 2012-5.csv 
\
----------| validation.csv 

### Schema de tabla en MySQL:
Antes de realizar el proyecto que se muestra en el notebook se creo una base de datos y una tabla basada en el schema que vi conveniente para realizar el proyecto.

CREATE SCHEMA `pragma_prueba`;

CREATE TABLE `events` (
\
  `event_id` int NOT NULL AUTO_INCREMENT,
\
  `timestamp` timestamp NOT NULL,
\
  `price` double DEFAULT NULL,
\
  `user_id` int NOT NULL,
\
  `filename` varchar(20) NOT NULL,
\
  PRIMARY KEY (`event_id`)
\  
) ENGINE=InnoDB AUTO_INCREMENT=2030 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci

##### Donde:
* `event_id`: llave primaria, no nula, entera y auto incrementable.
* `timestamp`: columna temporal en formato TIMESTAMP, no nula.
* `price`: columna definida como double, puede ser nula.
* `user_id`: columna que representa el id del usuario, entera y no nula.
* `filename`: columna que indica el nombre del archivo de donde se extrajo la informacion, tipo texto con 20 caracteres como limite, no nula.


# Librerias utilizadas en Python

In [10]:
import pymysql # libreria que sirve para conectar MySQL y realizar consultas y en general usar todo el lenguaje SQL desde Python3
import pandas as pd # libreria que se utilizo para manipulacion de tablas en Python y convercion a tipos TIMESTAMP
import numpy as np # libreria que se utilizo para manipular arrays en Python.
import os # liberia que se utilizo para adquirir listado de archivos dentro de la carpeta del proyecto.

import warnings # permite que no aparezca warnings especificos.
from pandas.core.common import SettingWithCopyWarning # en este caso se evita el warning de tipo SettingWithCopyWarning 
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning) # que es debido a un modulo desactualizado de la libreria pandas.

### Codigo:

In [3]:
archivo = os.listdir('dataPruebaDataEngineer/') # Obtiene una lista de los archivos dentro de la carpeta dataPruebaDataEngineer
not_included = ['validation.csv'] # Se genera una lista de documentos que no se desean evaluar.
for i in not_included:
    archivo.remove(i) # remueve los elementos de la lista que se evaluara.

In [21]:
def pipeline_prueba(archivo):
    '''
    Funcion que permite conexion con MySQL, carga los datos, los almacena en la base de datos previamente construida y con los
    datos almacenados en memoria se realiza un estudio de seguimiento respecto a:
            - Count o recuento de eventos durante el mes
            - Valor medio o promedio del mes
            - Valor minimo de la columna price durante el mes
            - Valor maximo de la columna price durante el mes
    Finalmente imprime los resultados obtenidos del seguimiento y se observa en la base de datos: "pragma_prueba" los datos almacenados.
    '''
    db = pymysql.connect( # conexion con host local creado para pruebas.
            host='localhost', # nombre del host local
            user='root', # usuario
            password='root', # contraseña
            db='pragma_prueba' # base de datos
        )

    cursor = db.cursor() # construye un cursor que permite realizar los query en MySQL
 
    for i in range(len(archivo)): # bucle que se mueve en el rango de la longitud de los archivos a evaluar
        
        temp = pd.read_csv(f"dataPruebaDataEngineer/{archivo[i]}") # carga los datos de cada archivo en la secuencia del bucle.
        temp_df = temp.copy() # realiza una copia de los datos.
        temp_df["filename"] = archivo[i] # crea la columna filename que permite almacenar el nombre del archivo de donde se extrae la informacion.
        temp_df.fillna(0,inplace=True) # Debido a que python no permite caracteres de tipo null y el mas similar es nan que no lo permite sql
        # ademas de que los null se encuentran en price, se decidio tomar esos valores como 0 debido a que no aportan para los resultados del promedio.
        for j in range(len(temp_df)): # bucle que se mueve en cada dato de la columna de timestamp
            temp_df.timestamp[j] = pd.Timestamp(temp_df.timestamp[j]) # permite a cada dato de timestamp cambiarlo de string a type(TIMESTAMP)
        
        values = temp_df.to_numpy() # recolecta los datos de cada fila de la carga de archivos.
        sql = "INSERT INTO events(timestamp,price,user_id,filename) VALUES "+",".join("(%s,%s,%s,%s)" for _ in values) # realiza INSERT INTO para agregar los
        # datos en la tabla construida en MySQL, envia en una sola linea todos los datos recolectados.
        fl_v = [item for sublist in values for item in sublist] # flatten de la variable values que permite usar los datos recolectados.

        cursor.execute(sql,fl_v) # ejecucion de sentencia SQL, donde se especifica la consulta y los datos.
        db.commit() # commit de la sentencia SQL


        #Debido que piden no utilizar los datos de la base de datos sino los almacenados en memoria
        # se realiza el seguimiento de estadistica a los valores almacenados en memoria que se encuentran en temp_df.
        count_events = temp_df.price.count() # cuenta cuantos eventos hubo en el mes
        temp_price = temp_df[temp_df.price != 0.0] # debido a que 0.0 no es el minimo sino un numero vacio, se toman solo los valores que no sean 0.0 en price
        min_price = temp_price.price.min() # mide el minimo valor de price durante el mes.
        max_price = temp_df.price.max() # mide el maximo valor de price durante el mes.
        prom_price = temp_df.price.mean() # mide el promedio del valor price durante el mes.
        month = temp_df.timestamp[0].month # observa el mes especificamente en que se realiza la consulta.
        
        # imprime el seguimiento de los datos.
        print(" Numero de eventos en el mes", month,":", count_events, "\n", 
            "Minimo valor de price: ", min_price, "\n",
            "Maximo valor de price: ", max_price, "\n",
            "Valor medio de price: ", prom_price )
        print("---------------------------------------\n")
        

    db.close() # Una vez finalizado el almacenado en la base de datos, se cierra la base de datos por seguridad.

In [27]:
pipeline_prueba(archivo) # analisis de los datos almacenados de enero a mayo, es decir, los archivos de 1 a 5.

 Numero de eventos en el mes 1 : 22 
 Minimo valor de price:  14.0 
 Maximo valor de price:  97.0 
 Valor medio de price:  54.22727272727273
---------------------------------------

 Numero de eventos en el mes 2 : 29 
 Minimo valor de price:  10 
 Maximo valor de price:  100 
 Valor medio de price:  54.827586206896555
---------------------------------------

 Numero de eventos en el mes 3 : 31 
 Minimo valor de price:  12 
 Maximo valor de price:  99 
 Valor medio de price:  59.67741935483871
---------------------------------------

 Numero de eventos en el mes 4 : 30 
 Minimo valor de price:  10.0 
 Maximo valor de price:  97.0 
 Valor medio de price:  53.56666666666667
---------------------------------------

 Numero de eventos en el mes 5 : 31 
 Minimo valor de price:  13 
 Maximo valor de price:  100 
 Valor medio de price:  58.25806451612903
---------------------------------------



In [26]:
pipeline_prueba(['validation.csv']) # analisis de los datos almacenados en el conjunto de validacion.

 Numero de eventos en el mes 6 : 8 
 Minimo valor de price:  11 
 Maximo valor de price:  92 
 Valor medio de price:  41.75
---------------------------------------



# Codigo extra:
permite pedir al usuario que mes desea evaluar y realiza un select desde la base de datos, es decir que evalua directamente de la base de datos.

In [28]:
db = pymysql.connect( # conecta a la base de datos
    host='localhost',
    user='root',
    password='root',
    db='pragma_prueba'
)

cursor = db.cursor() # crea el cursos para obtener consultas desde python en MySQL

inp = int(input()) # pide al usuario un valor entero
if inp <= len(archivo): sql = "SELECT * FROM events WHERE filename = '2012-{}.csv'".format(inp) # si el valor se encuentra en la longitud de los archivos
# realiza la consulta.

cursor.execute(sql) # ejecuta el select, que trae todos los datos especificados en el filename
results = cursor.fetchall() # guarda los resultados de la consulta.

event = [] # lista vacia que almacena los datos selecionados.
for row in results: # recorre cada fila de los resultados por columna dentro de la base de datos.
    event_id = row[0] 
    timestamp = row[1]
    price = row[2]
    user_id = row[3]
    filename = row[4]

    event.append([event_id,timestamp,price,user_id,filename]) # Almacena los datos en la lista vacia.

df1 = pd.DataFrame(event,columns=["event_id","timestamp","price","user_id","filename"]) # crea un dataframe que contenga los eventos.
count_events = df1.price.count() # cuenta el numero de eventos
temp_price = df1[df1.price != 0.0] # debido a que 0.0 no es el minimo sino un numero vacio, se toman solo los valores que no sean 0.0 en price
min_price = temp_price.price.min() # mide el valor minimo de price en el mes
max_price = df1.price.max() # mide el valor maximo de price en el mes
prom_price = df1.price.mean() # mide el valor promedio de price en el mes
month = df1.timestamp[0].month # observa el mes especificamente en que se realiza la consulta.

print(" Numero de eventos en el mes", month,":", count_events, "\n", 
            "Minimo valor de price: ", min_price, "\n",
            "Maximo valor de price: ", max_price, "\n",
            "Valor medio de price: ", prom_price )
print("---------------------------------------\n")
    

db.close() # cierra la base de datos.

 Numero de eventos en el mes 1 : 22 
 Minimo valor de price:  0.0 
 Maximo valor de price:  97.0 
 Valor medio de price:  54.22727272727273
---------------------------------------

